In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [2]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import xgboost as xgb

In [3]:
# データの読み込み
train = pd.read_csv('../kaggledata/train.csv')

X = train[['GrLivArea', 'YearBuilt']]
y = train[['SalePrice']]

# 対数変換
X = np.log(X).values
y = np.log(y).values

# 標準化
sc = StandardScaler()
sc.fit_transform(X)

# 分割
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

# 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

## ・ベースラインモデル

In [19]:
# SVM
svr = SVR().fit(X_train, y_train)
svr_pred = svr.predict(X_test)
svr_mse = mean_squared_error(y_test, svr_pred)

# 決定木
dt = DecisionTreeRegressor(max_depth=5).fit(X_train, y_train)
dt_pred = dt.predict(X_test)
dt_mse = mean_squared_error(y_test, dt_pred)

# 線形回帰
lr = LinearRegression().fit(X_train, y_train)
lr_pred = lr.predict(X_test).ravel()
lr_mse = mean_squared_error(y_test, lr_pred)

# xgboost（参考）
xg = xgb.XGBRegressor().fit(X_train, y_train)
xg_pred = xg.predict(X_test).ravel()
xg_mse = mean_squared_error(y_test, xg_pred)

# 評価
print(f'SVM：{svr_mse}')
print(f'決定木：{dt_mse}')
print(f'線形回帰：{lr_mse}')
print(f'XBG：{xg_mse}')

[17:37:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SVM：0.04395091582650826
決定木：0.042680977503983995
線形回帰：0.04590550537802906
XBG：0.03358922969505076


## ① SVM + 決定木 + 線形回帰

In [5]:
pattern1 = (svr_pred + dt_pred + lr_pred) / 3
pattern1 = mean_squared_error(y_test, pattern1)
pattern1

0.03966782950842762

## ② SVM + 決定木

In [6]:
pattern2 = (svr_pred + dt_pred) / 2
pattern2 = mean_squared_error(y_test, pattern2)
pattern2

0.03881215108581243

## ③ 決定木 + 線形回帰

In [7]:
pattern3 = (dt_pred + lr_pred) / 2
pattern3 = mean_squared_error(y_test, pattern3)
pattern3

0.0412580005432294

## ④ SVM : 決定木 = 2 : 3 

In [8]:
pattern4 = (svr_pred*2 + dt_pred*3) / 5
pattern4 = mean_squared_error(y_test, pattern4)
pattern4

0.038865309076737305

>各モデルの推定値を平均するだけでベースラインより良いスコアがでた。

# 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

## 決定木でバギングを検証する。
### ベースラインスコア 決定木：0.042680977503983974

In [9]:
def bagging(num, sample_num):
    '''
    バギングを行う関数
    '''
    num = num #学習回数
    sample_num = sample_num #サンプル数
    pred = 0 #推定の初期値

    for _ in range(num):
        # ランダムにsample_num個抽出（重複あり）
        idx = np.random.choice(len(X_train), sample_num)

        # 学習・推定
        model = DecisionTreeRegressor(max_depth=5).fit(X_train[idx], y_train[idx])
        pred += model.predict(X_test) / num
    
    # 評価
    mse = mean_squared_error(y_test, pred)
    
    return mse, pred

In [10]:
# 700サンプルで学習を30回行った推定値の平均値に対するMSE
mse, pred = bagging(30, 700)
print(f'y_test ：{y_test[:5].ravel()}')
print(f'bagging：{pred[:5]}')
print(f'MSE    ：{mse}')

y_test ：[12.20918779 11.79810441 11.60823564 12.16525065 11.38509209]
bagging：[12.2414099  11.92949516 11.70797822 12.22877304 11.77793144]
MSE    ：0.036561018097023855


### ・本家ランダムフォレストでは・・・

In [11]:
rdt = RandomForestRegressor(max_depth=5).fit(X_train, y_train)
rdt_pred = rdt.predict(X_test)
rdt_mse = mean_squared_error(y_test, rdt_pred)
rdt_mse

0.035758140977409314

>ベースラインスコアを上回るスコアが得られ、決定木のバギングモデルであるランダムフォレストと同等のスコアであった。

# 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

In [12]:
def stacking_data(X, y, test):
    '''
    メタモデル用の学習データと検証データを取得する関数
    '''
    meta_train = np.zeros((len(X), 3)) #学習データの空箱
    meta_test = np.zeros((len(test), 3)) #検証データの空箱
    n_splits = 6 #分割数
    
    kf = KFold(n_splits=n_splits)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # 学習
        model1 = SVR().fit(X_train , y_train)
        model2 = DecisionTreeRegressor(max_depth=5).fit(X_train , y_train)
        model3 = LinearRegression().fit(X_train , y_train)

        # 学習データを推定
        pred_train_model1 = model1.predict(X_test).reshape(-1, 1)
        pred_train_model2 = model2.predict(X_test).reshape(-1, 1)
        pred_train_model3 = model3.predict(X_test).reshape(-1, 1)
        
        # 検証データを推定
        pred_test_model1 = model1.predict(test).reshape(-1, 1)
        pred_test_model2 = model2.predict(test).reshape(-1, 1)
        pred_test_model3 = model3.predict(test).reshape(-1, 1)
        
        # メタモデル学習用データを保管
        meta_train[test_index, :1] = pred_train_model1
        meta_train[test_index, 1:2] = pred_train_model2
        meta_train[test_index, 2:] = pred_train_model3
        
        # メタモデル検証用データを保管
        meta_test[:, :1] += pred_test_model1 / n_splits
        meta_test[:, 1:2] += pred_test_model2 / n_splits
        meta_test[:, 2:] += pred_test_model3 / n_splits
    
    return meta_train, meta_test

In [13]:
# メタモデル用の学習・検証データ取得
meta_train, meta_test = stacking_data(X_train, y_train, X_test)

# 学習
meta_model = SVR().fit(meta_train, y_train)

# 推定
pred_meta = meta_model.predict(meta_test)

# 評価
print(f'ステージ0：{mean_squared_error(y_test, pred_meta)}')

ステージ0：0.033658402413554854


>ここまでの検証の中で最高スコアであった。

In [14]:
for i in range(9):
    # メタモデル用の学習・検証データ取得
    meta_train, meta_test = stacking_data(meta_train, y_train, meta_test)

    # 学習
    meta_model = SVR().fit(meta_train, y_train)

    # 推定
    pred_meta = meta_model.predict(meta_test)

    # 評価
    print(f'ステージ{i+1}：{mean_squared_error(y_test, pred_meta)}')

ステージ1：0.035145494896062335
ステージ2：0.03505129747264169
ステージ3：0.03686269821979252
ステージ4：0.03750921839855495
ステージ5：0.037145348301597145
ステージ6：0.039482202011180155
ステージ7：0.04767650589529238
ステージ8：0.03810771911977699
ステージ9：0.04312807196575073


>ステージ数を増やすと精度が悪くなる傾向であった。